# Práctica de MapReduce sobre dataset de Juegos en Red

Este cuaderno contiene los ejercicios prácticos para aplicar patrones MapReduce sobre un dataset de sesiones de juegos online.

## **Alumno:** Rafael Navarro Gómez

### ⚙️ Configuración inicial del entorno Hadoop

> **Nota importante:**
> Antes de comenzar cada ejercicio, **cambia el valor de la variable `NAME_E`** para asignarle un nombre identificativo (por ejemplo: `01_wordcount`, `02_avg_duration`, etc.).
> Esto permitirá que **cada ejercicio se ejecute en un subdirectorio diferente dentro de HDFS**, evitando conflictos y conservando los resultados anteriores.
> Tras modificar `NAME_E`, **vuelve a ejecutar esta celda completa** para que se actualicen las rutas y variables del entorno.
> ⚠️ No modifiques las demás variables a menos que se indique expresamente.

In [49]:
# ==========================================================
# 🧩 CONFIGURACIÓN DEL ENTORNO DE TRABAJO PARA MAPREDUCE
# ==========================================================

# 🔹 Nombre del ejercicio actual (personalizable para cada enunciado)
NAME_E = '05'

# 🔹 Directorio base de trabajo local en Jupyter
WORK_DIR = '/home/jovyan/work'

# 🔹 Carpeta donde se encuentran los materiales proporcionados
MATERIALES = f'{WORK_DIR}/materiales'

# 🔹 Directorios de entrada y salida en HDFS
HDFS_BASE = '/user/shared'
HDFS_IN = f'{HDFS_BASE}/{NAME_E}'
HDFS_OUT = f'{HDFS_BASE}/{NAME_E}/salida'

# 🔹 Directorios locales equivalentes para trabajar temporalmente
LOCAL_IN = f'{WORK_DIR}{HDFS_IN}'
LOCAL_OUT = f'{WORK_DIR}{HDFS_OUT}'

# ==========================================================
# Declaración de variables de entorno para Hadoop y shell
# ==========================================================
%env MATERIALES=$MATERIALES
%env HDFS_IN=$HDFS_IN
%env HDFS_OUT=$HDFS_OUT
%env LOCAL_IN=$LOCAL_IN
%env LOCAL_OUT=$LOCAL_OUT

# Creamos los directorios locales necesarios (si no existen)
!mkdir -p $LOCAL_IN

# Cambiamos el directorio de trabajo actual
%cd $LOCAL_IN

# ==========================================================
# 🧹 (Opcional) Limpieza de ejecuciones previas en HDFS
# ==========================================================
# ⚠️ Descomenta la siguiente línea si necesitas eliminar resultados anteriores
# ! hdfs dfs -rm -r -f $HDFS_IN

env: MATERIALES=/home/jovyan/work/materiales
env: HDFS_IN=/user/shared/05
env: HDFS_OUT=/user/shared/05/salida
env: LOCAL_IN=/home/jovyan/work/user/shared/05
env: LOCAL_OUT=/home/jovyan/work/user/shared/05/salida
/home/jovyan/work/user/shared/05


### 📂 Carga del dataset

Sube el archivo **`game_sessions_1Mi.csv`** que se proporciona junto a este notebook al entorno de Jupyter usando el botón de `Upload`.  
Una vez subido, comprueba que se muestra en el directorio de trabajo:

In [2]:
!echo '📁 Contenido de la carpeta de materiales:'
!ls -l $MATERIALES
!echo ''
!echo '📁 Copiar el dataset en HDFS para que esté disponible en todos los ejercicios:'
! hdfs dfs -put $MATERIALES/game_sessions_1Mi.csv $HDFS_BASE/

📁 Contenido de la carpeta de materiales:
total 35204
-rw-r--r-- 1 jovyan users 36047049 Nov 20 18:32 game_sessions_1Mi.csv

📁 Copiar el dataset en HDFS para que esté disponible en todos los ejercicios:
put: `/user/shared/game_sessions_1Mi.csv': File exists


### 📊 Introducción al dataset

El dataset `game_sessions_1Mi.csv` contiene registros de sesiones de juegos online.  
Cada fila representa una partida individual de un usuario, incluyendo información sobre:
- Título del juego (`game_title`)
- Región del jugador (`region`)
- Plataforma utilizada (`platform`)
- Duración de la sesión (`session_duration_min`)
- Puntuación obtenida (`score`)
- Objetos recolectados (`items_collected`)
- Resultado del equipo o jugador (`team_result`)

Este dataset se usará para aplicar distintos patrones MapReduce.

### 🧩 Bloque de ejercicios prácticos

> **Indicaciones generales:**
> - En cada ejercicio, muestra **el código completo de Mapper y Reducer** y los **resultados de la ejecución**.
> - Puedes añadir celdas auxiliares para comprobaciones intermedias.
> - Cada ejercicio se ejecutará en un **subdirectorio independiente en HDFS**.
> - Incluye siempre una **celda final de verificación de resultados en HDFS** al final de cada ejercicio.
> - Las celdas con errores de ejecución o salidas incompletas serán penalizadas.

### Ejercicio 1 – Conteo de sesiones por juego

En este ejercicio tienes que calcular el **número total de sesiones por cada juego**, usando MapReduce.  
Cada fila del dataset representa una sesión individual de un usuario.

In [40]:
%%writefile mapper.py
import sys

header_line = True

data = sys.stdin

for line in data:
    if header_line:
        header_line = False
        continue
    line = line.strip()
    line = line.split(',')

    game_name = line[1]
    
    print(f"{game_name},{1}")

Overwriting mapper.py


In [41]:
%%writefile reducer.py

import sys

# variables para los valores de cada iteración
sessions_per_game = {}

# entrada desde STDIN
for line in sys.stdin:
    # eliminamos espacios blancos al principio y final
    line = line.strip()

    game, count = line.split(",")
    
    try:
        count = int(count)
    except ValueError:
        continue

    if game in sessions_per_game:

        sessions_per_game[game] += 1

    else:

        sessions_per_game[game] = 1

for game, times in sessions_per_game.items():
    print(f"{game}, {times}")


Overwriting reducer.py


In [42]:
# Comando de ejecución en Hadoop
!cat $MATERIALES/game_sessions_1Mi.csv | python3 mapper.py | python3 reducer.py

FIFA, 325276
Minecraft, 81340
Valorant, 162440
Call of Duty, 40489
Fortnite, 64699
League of Legends, 113755
Overwatch, 65229
Rocket League, 73400
Among Us, 40772
Apex Legends, 32600


In [43]:
# Verificación de resultados en HDFS del ejercicio 1
!echo '📂 Contenido de HDFS de salida del ejercicio 1:'
!hdfs dfs -ls $HDFS_OUT
!hdfs dfs -cat $HDFS_OUT/part-* | sort -k2,2nr

📂 Contenido de HDFS de salida del ejercicio 1:
ls: `/user/shared/01/salida': No such file or directory
cat: `/user/shared/01/salida/part-*': No such file or directory


### Ejercicio 2 – Duración media por región

Debes calcular la **duración media de las sesiones** para cada región de Europa.

In [5]:
%%writefile mapper.py

import sys

header_line = True

data = sys.stdin

for line in data:
    if header_line:
        header_line = False
        continue
    line = line.strip()
    line = line.split(',')

    game_name = line[1]
    mins_played = line[4]
    
    print(f"{game_name},{mins_played}")

Overwriting mapper.py


In [48]:
%%writefile reducer.py

import sys

# variables para los valores de cada iteración
sessions_per_game = {}

# entrada desde STDIN
for line in sys.stdin:
    # eliminamos espacios blancos al principio y final
    line = line.strip()

    game, count = line.split(",")
    
    try:
        count = int(count)
    except ValueError:
        continue

    if game in sessions_per_game:

        sessions_per_game[game].append(count)

    else:
        sessions_per_game[game] = [count]

for game, times in sessions_per_game.items():
    total_mins = sum(times)
    media_mins = total_mins / len(times)
    print(f"{game}, {media_mins}")


Overwriting reducer.py


In [22]:
# Comando de ejecución en Hadoop
!cat $MATERIALES/game_sessions_1Mi.csv | python3 mapper.py | python3 reducer.py | tail


197, 95.4245283018868
694, 99.20992761116857
299, 101.50324675324676
52, 226.3980044345898
71, 222.07841031149303
765, 99.57049180327868
671, 98.39957035445757
136, 100.51859723698193
150, 100.1659090909091
808, 101.66203703703704


In [50]:
# Verificación de resultados en HDFS del ejercicio 2
!echo '📂 Contenido de HDFS de salida del ejercicio 2:'
!hdfs dfs -ls $HDFS_OUT
!hdfs dfs -cat $HDFS_OUT/part-* | sort -k2,2nr

📂 Contenido de HDFS de salida del ejercicio 2:
ls: `/user/shared/02/salida': No such file or directory
cat: `/user/shared/02/salida/part-*': No such file or directory


### Ejercicio 3 – Puntuación media por jugador

Determina la **puntuación media de cada jugador (`user_id`)** a lo largo de sus partidas.

In [15]:
%%writefile mapper.py

import sys

header_line = True

data = sys.stdin

for line in data:

    if header_line:
        header_line = False
        continue
        
    line = line.strip()
    line = line.split(',')

    user_id = line[0]
    score = line[5]

    print(f"{user_id},{score}")

Overwriting mapper.py


In [17]:
%%writefile reducer.py

import sys

# variables para los valores de cada iteración
user_avg_score = {}

# entrada desde STDIN
for line in sys.stdin:
    # eliminamos espacios blancos al principio y final
    line = line.strip()

    user, count = line.split(",")
    
    try:
        count = int(count)
    except ValueError:
        continue

    if user in user_avg_score:

        user_avg_score[user].append(count)

    else:
        user_avg_score[user] = [count]

for game, times in user_avg_score.items():
    total_mins = sum(times)
    media_mins = total_mins / len(times)
    print(f"{game}, {media_mins}")

Writing reducer.py


In [23]:
# Comando de ejecución en Hadoop
!cat $MATERIALES/game_sessions_1Mi.csv | python3 mapper.py | python3 reducer.py | tail


197, 95.4245283018868
694, 99.20992761116857
299, 101.50324675324676
52, 226.3980044345898
71, 222.07841031149303
765, 99.57049180327868
671, 98.39957035445757
136, 100.51859723698193
150, 100.1659090909091
808, 101.66203703703704


In [ ]:
# Verificación de resultados en HDFS del ejercicio 3
!echo '📂 Contenido de HDFS de salida del ejercicio 3:'
!hdfs dfs -ls $HDFS_OUT
!hdfs dfs -cat $HDFS_OUT/part-* | sort -k2,2nr | head -10

### Ejercicio 4 – Top 5 jugadores con más victorias

Identifica los **5 jugadores con más victorias** (`team_result='WIN'`).

In [25]:
%%writefile mapper.py

import sys

header_line = True

data = sys.stdin

for line in data:

    if header_line:
        header_line = False
        continue

    line = line.strip()
    line = line.split(',')

    user = line[0]
    result = line[7]

    print(f"{user},{result}")


Writing mapper.py


In [47]:
%%writefile reducer.py

import sys

top_scores = {}

data = sys.stdin

for line in data:

    line = line.strip()
    user, result = line.split(',')

    if user in top_scores:
        top_scores[user] += 1

    else:
        top_scores[user] = 1

sorted_top_scores = dict(sorted(top_scores.items(), key=lambda x: x[1], reverse=True)[:5])

print(sorted_top_scores)


Overwriting reducer.py


In [48]:
# Comando de ejecución en Hadoop
!cat $MATERIALES/game_sessions_1Mi.csv | python3 mapper.py | python3 reducer.py


{'10': 12613, '9': 11645, '8': 10572, '7': 9427, '6': 8522}


In [ ]:
# Verificación de resultados en HDFS del ejercicio 4
!echo '📂 Contenido de HDFS de salida del ejercicio 4:'
!hdfs dfs -ls $HDFS_OUT
!hdfs dfs -cat $HDFS_OUT/part-* | sort -k2,2nr | head -5

### Ejercicio 5 – Distribución por plataforma y resultado

Calcula la **distribución de partidas por plataforma y resultado**.

In [50]:
%%writefile mapper.py

import sys

header_line = True

data = sys.stdin

for line in data:

    if header_line:
        header_line = False
        continue

    line = line.strip()
    line = line.split(',')

    platform = line[3]
    result = line[7]

    print(f"{platform},{result}")


Writing mapper.py


In [58]:
%%writefile reducer.py

import sys

def checkIfIsThere(dict, value):
    if value in dict:
        dict[value] += 1

    else:
        dict[value] = 1

data = sys.stdin

platforms_results = {}

for line in data:

    line = line.strip()
    platform, result = line.split(',')

    if platform in platforms_results:
        checkIfIsThere(platforms_results[platform], result)
    else:
        platforms_results[platform] = {}
        checkIfIsThere(platforms_results[platform], result)

print(platforms_results)

Overwriting reducer.py


In [59]:
# Comando de ejecución en Hadoop
!cat $MATERIALES/game_sessions_1Mi.csv | python3 mapper.py | python3 reducer.py | tail


{'PC': {'LOSE': 180324, 'DRAW': 180003, 'WIN': 240195}, 'Console': {'WIN': 100087, 'DRAW': 74821, 'LOSE': 75269}, 'Mobile': {'LOSE': 44688, 'WIN': 59827, 'DRAW': 44786}}


In [ ]:
# Verificación de resultados en HDFS del ejercicio 5
!echo '📂 Contenido de HDFS de salida del ejercicio 5:'
!hdfs dfs -ls $HDFS_OUT
!hdfs dfs -cat $HDFS_OUT/part-* | sort -k2,2nr

Una vez que hayas terminado todos los ejercicios, recuerda **guardar tu cuaderno de Jupyter** y descargarlo a tu ordenador.  
Después, sube el archivo a la plataforma online para entregar tu trabajo.  
**No olvides** incluir tu nombre y apellidos al pricipio de este documento.